In [1]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

translating es to en
callhome es-en word level configuration
vocab size, en=5691, fr=8904


In [2]:
xp = cuda.cupy if gpuid >= 0 else np

In [3]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [4]:
if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()

In [ ]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [ ]:
log_train_fil_name, text_fname, dev_fname, test_fname

In [ ]:
%aimport nmt_trials
from nmt_trials import *

In [ ]:
buckets, bucket_lengths = populate_buckets()

In [ ]:
25*32, 24*35

In [ ]:
_, en, sf1 = get_data_item('053.182')
_, en, sf2 = get_data_item('053.183')
xp.isnan(xp.sum(sf))

In [ ]:
sf1.shape, sf2.shape

In [ ]:
_, _, sp1 = get_data_item("041.002", "train")
_, _, sp2 = get_data_item("041.004", "train")
batch_data = [sp1[:100], sp2[:100]]
print(sp1.shape, sp2.shape)

In [ ]:
# from chainer.utils import type_check
# from chainer import function
# class PadSequence(function.Function):

#     def __init__(self, length, padding):
#         self.length = length
#         self.padding = padding

#     def check_type_forward(self, in_types):
#         type_check.expect(in_types.size() > 0)

#         for in_type in in_types:
#             type_check.expect(
#                 in_type.ndim > 0,
#                 in_type.shape[1:] == in_types[0].shape[1:],
#                 in_type.dtype == in_types[0].dtype)

#         if self.length is not None:
#             for in_type in in_types:
#                 type_check.expect(in_type.shape[0] <= self.length)

#     def forward(self, xs):
#         xp = cuda.get_array_module(*xs)

#         if self.length is None:
#             length = max(len(x) for x in xs)
#         else:
#             length = self.length

#         shape = (len(xs), length) + xs[0].shape[1:]
#         y = xp.empty(shape, xs[0].dtype)
#         if length == 0:
#             return y,

#         if xp is np or any(not x._c_contiguous for x in xs):
#             for i, x in enumerate(xs):
#                 l = len(x)
#                 if l == length:
#                     y[i] = x
#                 else:
#                     y[i, 0:l] = x
#                     y[i, l:] = self.padding
#         else:
#             # This code assumes that all arrays are c_contiguous
#             ptr_shape = (Ellipsis,) + (None,) * xs[0].ndim
#             ptrs = cuda.cupy.array([x.data for x in xs], 'L')[ptr_shape]
#             lengths = cuda.cupy.array([len(x) for x in xs], 'i')[ptr_shape]
#             base = np.prod(xs[0].shape[1:], dtype='i')
#             cuda.elementwise(
#                 'P ptr, int32 length, T pad, int32 base, int32 max_length',
#                 'T y',
#                 '''
#                 int d = i / base % max_length;
#                 if (d < length) {
#                   y = reinterpret_cast<const T*>(ptr)[i % (base * max_length)];
#                 } else {
#                   y = pad;
#                 }
#                 ''',
#                 'pad_sequence_fwd'
#             )(ptrs, lengths, self.padding, base, length, y)

#         return y,

#     def backward(self, xs, grad):
#         xp = cuda.get_array_module(*xs)
#         gs = grad[0]
#         if gs.size == 0:
#             # `split` in NumPy 1.9 behaves inconsistently when size is zero.
#             gs = [gs]
#         else:
#             gs = xp.split(gs, len(xs), axis=0)
#         return tuple([g[0, 0:len(x)] for g, x in six.moves.zip(gs, xs)])


# def pad_sequence(xs, length=None, padding=0):
#     """Pad given arrays to make a matrix.
#     Args:
#         xs (list of ~chainer.Variable): Variables you want to concatenate.
#         length (None or int): Size of the first dimension of a padded array.
#             If it is ``None``, the longest size of the first dimension of
#             ``xs`` is used.
#         padding (int or float): Value to fill.
#     Returns:
#         ~chainer.Variable: It returns a padded matrix. Its shape is
#             ``(n, length, ...)``, where ``n == len(xs)``.
#     """
#     return PadSequence(length, padding)(*xs)

In [ ]:
hahaha = xp.array([[1,2],[3,4],[5,6]], dtype=xp.float32)

In [ ]:
F.flipud(hahaha)

In [ ]:
haha = F.pad_sequence(xs=(batch_data), length=100, padding=0)

In [ ]:
haha[0].data

In [ ]:
haha = F.pad_sequence(ha)

In [ ]:
haha.shape

In [ ]:
buck_indx = 23
i = 0
batch_size = 50

pad_size_speech = (buck_indx+1) * SPEECH_BUCKET_WIDTH
pad_size_en = min(bucket_lengths[buck_indx][3], MAX_EN_LEN)

src_lim = pad_size_speech
tar_lim = pad_size_en

sp_files_in_batch = [t[0] for t in buckets[buck_indx][i:i+batch_size]]

# get the next batch of data
batch_data = []
for sp_fil in sp_files_in_batch:
    _, en_ids, speech_feat = get_data_item(sp_fil, cat="train")
    # print(speech_feat.shape, len(en_ids))
    batch_data.append((speech_feat[:pad_size_speech], en_ids[:pad_size_en]))

# compute loss
# loss = model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=True)

In [ ]:
batch_size = len(batch_data)
in_shape_r, in_shape_c = batch_data[0][0].shape

fwd_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
rev_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
decoder_batch = xp.full((batch_size, pad_size_en+2), PAD_ID, dtype=xp.int32)

In [ ]:
for i, (src, tar) in enumerate(batch_data):
    fwd_encoder_batch[i] = model.pad_array(src, src_lim)
    rev_encoder_batch[i] = model.pad_array(xp.flip(src, axis=0), src_lim)

    tar_data = [GO_ID] + tar + [EOS_ID]
    decoder_batch[i] = model.pad_list(tar_data, tar_lim+2, at_start=False)

In [ ]:
train=True

In [ ]:
fwd_encoder_batch = xp.swapaxes(fwd_encoder_batch, 0,1)
rev_encoder_batch = xp.swapaxes(rev_encoder_batch, 0,1)
decoder_batch = xp.swapaxes(decoder_batch, 0,1)

In [ ]:
L_FWD_STATES = model.encode_speech_batch_lstm(fwd_encoder_batch, model.lstm_enc, train=True)

In [ ]:
L_REV_STATES = model.encode_speech_batch_lstm(rev_encoder_batch, model.lstm_rev_enc, train)
# reverse the states to align them with forward encoder
L_REV_STATES = xp.flip(L_REV_STATES, axis=0)

In [ ]:
return_shape = L_FWD_STATES.shape
model.enc_states = F.concat((L_FWD_STATES, L_REV_STATES), axis=2)
model.enc_states = F.swapaxes(model.enc_states, 0, 1)

In [ ]:
model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=train)

In [ ]:
L_FWD_STATES.shape

In [ ]:
pad_size_speech / 8

In [ ]:
in_size, batch_size, in_dim = fwd_encoder_batch.shape

In [ ]:
print(in_size, batch_size, in_dim)

In [ ]:
speech_feats = xp.load(os.path.join(speech_dir, "train.npz"))

In [ ]:
haha = speech_feats["041.001"]

In [ ]:
haha.shape

In [ ]:
haha = xp.array([[2,-2], [1,-1], [3,-3]], dtype=xp.float32)

In [ ]:
haha

In [ ]:
meh = F.expand_dims(haha.swapaxes(0,1), axis=0)

In [ ]:
meh.shape

In [ ]:
def cnn_out_size(i, p, k):
    o = (i-k) + 2*p + 1
    print("cnn out = {0:d}".format(o))

In [ ]:
k_width = 2
pad = k_width // 2
# pad = k_width - 1
pad = 1

In [ ]:
if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()
c1 = L.ConvolutionND(ndim=1, in_channels=SPEECH_DIM, out_channels=1, ksize=k_width, pad=pad).to_gpu()

In [ ]:
c1.W.shape

In [ ]:
i = meh.shape[2]
out_shape = (i-k_width) + 2*pad + 1
print(i, out_shape)

In [ ]:
%%timeit -n1 -r2
ha = c1(meh)
print(ha.shape)
print(ha.data)

### test with small arrays

In [ ]:
mini_test = xp.array([[2,-2,-5], [1,-1,-10], [3,-3,-9], [4,-4,-12]], dtype=xp.float32)

In [ ]:
mini_test, mini_test.shape

In [ ]:
mini_var = F.expand_dims(mini_test.swapaxes(0,1), axis=0)

In [ ]:
mini_var.shape

In [ ]:
mini_k = 3; mini_pad = mini_k // 2
minic1 = L.ConvolutionND(ndim=1, in_channels=3, out_channels=5, ksize=2, stride=1, pad=1).to_gpu()

In [ ]:
cnn_out_size(i=4, k=2, p=1)

In [ ]:
print(minic1.W.shape)
print(minic1.W.data)

In [ ]:
one = minic1.W.data[0]
one.shape

In [ ]:
mini_test[0], mini_test[1]

In [ ]:
h1 = one[0][0]*mini_test[0][0]
h2 = one[1][0]*mini_test[0][1]
h3 = one[2][0]*mini_test[0][2]
h4 = one[0][1]*mini_test[1][0]
h5 = one[1][1]*mini_test[1][1]
h6 = one[2][1]*mini_test[1][2]

In [ ]:
sum([h1, h2, h3, h4, h5, h6])

In [ ]:
mini_cnn_out = minic1(mini_var).data

In [ ]:
mini_cnn_out.shape, mini_cnn_out

In [ ]:
F.max_pooling_nd(mini_cnn_out, ksize=2, stride=1, pad=mini_pad).data

In [ ]:
speech_feats = {}
speech_feats['train'] = xp.load(os.path.join(speech_dir, "train.npz"))

In [ ]:
sp_1 = speech_feats['train']['041.004']
print(len(sp_1)), sp_1.shape

In [ ]:
CHAR_LEVEL

In [ ]:
en_words = [w.word for w in text_data['train']['041.004']['en']]
en_symbols = [w.encode() for w in en_words]
en_ids = [w2i["en"].get(w, UNK_ID) for w in en_symbols]

In [ ]:
en_ids_array = xp.array(en_ids, dtype=xp.int32)

In [ ]:
F.pad_sequence([en_ids_array, en_ids_array], length=20)

In [ ]:
pad_width = 16

In [ ]:
sp1_pad = xp.pad(sp_1, ((pad_width - len(sp_1) % pad_width,0),(0,0)), mode='constant')

In [ ]:
len(sp1_pad), len(sp_1) % pad_width, len(sp1_pad) % pad_width, sp1_pad.shape

In [ ]:
sp1_pad.shape

In [ ]:
sp_1[0,:5], sp1_pad[:20,:5]

In [ ]:
sp1_pad.shape

In [ ]:
empty_t = xp.empty(shape=(0,SPEECH_DIM), dtype=xp.float32)

In [ ]:
empty_t = xp.concatenate((empty_t, sp1_pad), axis=0)
print(empty_t.shape)
empty_t = xp.concatenate((empty_t, sp1_pad), axis=0)
print(empty_t.shape)

In [ ]:
sp1_pad_rev = xp.flipud(sp_1)

In [ ]:
sp1_pad_rev[:3,:3], sp1_pad[-3:,:3]

In [121]:
%run nmt_trials.py

rnn_in_units 90
using ADAM optimizer
loading data
finished loading data
Starting experiment
mfcc_kaldi_GRU_again/dev_17394sen_1-2layers_100units_rnn_word_lstm_adam_callhome_es_en_1.log
mfcc_kaldi_GRU_again/seq2seq_17394sen_1-2layers_100units_rnn_word_lstm_adam_callhome_es_en_1.model
num sentences=10000 and num epochs=10
split data into 50 buckets, each of width=16


100%|██████████| 2476/2476 [00:04<00:00, 575.56it/s]


padding labels
11 items in bucket=1, each of length=39, max en ids=4
98 items in bucket=2, each of length=39, max en ids=4
188 items in bucket=3, each of length=39, max en ids=6
145 items in bucket=4, each of length=39, max en ids=6
148 items in bucket=5, each of length=39, max en ids=8
124 items in bucket=6, each of length=39, max en ids=8
105 items in bucket=7, each of length=39, max en ids=11
102 items in bucket=8, each of length=39, max en ids=9
102 items in bucket=9, each of length=39, max en ids=11
82 items in bucket=10, each of length=39, max en ids=11
94 items in bucket=11, each of length=39, max en ids=16
82 items in bucket=12, each of length=39, max en ids=16
69 items in bucket=13, each of length=39, max en ids=15
68 items in bucket=14, each of length=39, max en ids=15
73 items in bucket=15, each of length=39, max en ids=16
70 items in bucket=16, each of length=39, max en ids=17
52 items in bucket=17, each of length=39, max en ids=21
55 items in bucket=18, each of length=39, 

In [122]:
# buckets_dict['train'] = prepare_data()

In [123]:
# buckets_dict['train'][0]["X_fwd"].shape

In [124]:
len(buckets_dict['dev'][1]["X_fwd"])

98

In [125]:
t = buckets_dict['dev'][1]['X_fwd'][:20]

In [126]:
out1 = model.forward_cnn(t)

(20, 10, 32)
(20, 20, 32)
(20, 30, 32)
(20, 40, 32)
(20, 50, 32)
(20, 60, 32)
(20, 70, 32)
(20, 80, 32)
(20, 90, 32)
(20, 90, 8)


/afs/inf.ed.ac.uk/group/project/lowres/work/anaconda3/envs/chainer2/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.functions.pooling.MaxPoolingND is experimental. The interface can change in the future.
  FutureWarning)


In [127]:
h = F.rollaxis(out1, axis=2)
print(h.shape), 
print(model.scale)

(8, 20, 90)
1


In [128]:
hf, hr = model.forward_rnn(h)

X (8, 20, 90)


In [130]:
hr

variable([[[             nan,              nan,              nan, ...,
                         nan,              nan,              nan],
           [             nan,              nan,              nan, ...,
                         nan,              nan,              nan],
           [             nan,              nan,              nan, ...,
                         nan,              nan,              nan],
           ..., 
           [             nan,              nan,              nan, ...,
                         nan,              nan,              nan],
           [             nan,              nan,              nan, ...,
                         nan,              nan,              nan],
           [             nan,              nan,              nan, ...,
                         nan,              nan,              nan]],

          [[             nan,              nan,              nan, ...,
                         nan,              nan,              nan],
           [   

In [84]:
he = model.encode(h[0], model.rnn_enc)

In [85]:
hf

variable([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           ..., 
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

          [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           ..., 
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

          [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           ..., 
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  nan, ...,  nan,  nan,  nan],
           [ nan,  nan,  na

In [63]:
# model.L0_enc = model.RNN(90,100)
model.add_rnn_layers(['nL_0'], 90, model.n_units, scale=1)

In [99]:
model.to_gpu()

In [77]:
model.L0_enc(h[2])

variable([[ 0.17088197,  0.13062714, -0.06950388, ..., -0.14050233,
           -0.30183068, -0.19015318],
          [-0.00132584,  0.09462968,  0.0914233 , ...,  0.00338074,
           -0.15398282, -0.07348515],
          [ 0.20481794,  0.09833317,  0.27618104, ...,  0.01565787,
           -0.23376104, -0.11751523],
          ..., 
          [ 0.14942464,  0.11332168,  0.0842166 , ...,  0.10079847,
           -0.09364501,  0.0267418 ],
          [ 0.06728604,  0.07206888, -0.05080219, ...,  0.03351169,
           -0.22022687, -0.04394259],
          [ 0.05603104,  0.09172393,  0.0522528 , ..., -0.03993868,
           -0.26154986,  0.1199726 ]])

In [89]:
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    lstm_test = model.RNN(90, 100)
    lstm_test.to_gpu()

In [47]:
model.add_rnn_layers(in_units=90, out_units=100,layer_names=['rnn_test'], scale=1)

In [48]:
model.to_gpu()
model.rnn_test(h[2])

variable([[-0.06926221, -0.21527876, -0.04705128, ...,  0.27475366,
           -0.37026265,  0.06866559],
          [ 0.0130947 , -0.07874557, -0.01964537, ...,  0.02807949,
           -0.13835178,  0.0891204 ],
          [-0.04015987, -0.17219132,  0.05235179, ...,  0.18459854,
            0.05624923,  0.02781546],
          ..., 
          [-0.11278225, -0.04982725,  0.02328975, ...,  0.08394802,
            0.12310678,  0.03418106],
          [-0.04079617, -0.08743001, -0.1354847 , ...,  0.11196797,
            0.19609286, -0.07657846],
          [-0.21576719, -0.12321437,  0.06111503, ...,  0.21908043,
           -0.26039788,  0.03160518]])

In [98]:
lstm_test(h[2])

variable([[ 0.04944006,  0.09556714, -0.44437841, ..., -0.14781998,
            0.65636563,  0.19542442],
          [ 0.01253496,  0.00241905, -0.49145231, ...,  0.19214736,
            0.49585962,  0.11678124],
          [-0.12379218,  0.00652893, -0.39462811, ..., -0.18692628,
            0.74462467,  0.20249383],
          ..., 
          [ 0.16764103, -0.15657805, -0.192853  , ...,  0.28847459,
            0.29622197,  0.15147099],
          [-0.18047023, -0.10877994,  0.0760225 , ..., -0.16628039,
            0.40716025,  0.20774698],
          [ 0.12239774, -0.22550839, -0.53163111, ..., -0.10556503,
            0.29653063,  0.41799688]])

In [165]:
h

variable([[[ 0.48160893,  0.29182747,  0.        , ...,  0.37724853,
             0.35354695,  1.11942744],
           [ 0.        ,  0.        ,  0.        , ...,  0.90547466,
             0.6689232 ,  0.55247557],
           [ 0.07088542,  0.54246229,  0.        , ...,  0.        ,
             0.59680718,  0.20341606],
           ..., 
           [ 0.        ,  0.        ,  0.        , ...,  0.29700285,
             0.63800472,  0.34882778],
           [ 0.        ,  0.        ,  0.        , ...,  0.59988761,
             0.62842304,  0.        ],
           [ 0.10303846,  0.42879567,  0.67467916, ...,  1.26035666,
             0.36838549,  0.        ]],

          [[ 1.25348902,  0.02680092,  1.60453534, ...,  0.41225335,
             1.40000391,  0.02314951],
           [ 0.        ,  0.        ,  0.        , ...,  0.53784353,
             0.43993512,  0.145229  ],
           [ 2.03745151,  1.66896772,  0.        , ...,  0.33901104,
             0.60412908,  0.        ],
         

In [161]:
# model.L0_rev_enc.reset_state()
hs = model.L0_rev_enc(h[-1])
hs

variable([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
          [ nan,  nan,  nan, ...,  nan,  nan,  nan],
          [ nan,  nan,  nan, ...,  nan,  nan,  nan],
          ..., 
          [ nan,  nan,  nan, ...,  nan,  nan,  nan],
          [ nan,  nan,  nan, ...,  nan,  nan,  nan],
          [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [51]:
model[model.rnn_enc[0]]

In [166]:
xp.isnan(xp.sum(h.data)) == True

array(False, dtype=bool)

In [31]:
xp.isnan(xp.sum(model.enc_states.data)) == True

array(True, dtype=bool)

In [ ]:
# model.forward_highway(out1)

In [ ]:
out1 = model.CNN_9(t)
print(out1.shape)
out2 = model.CNN_19(t)
print(out2.shape)

combined_out = F.concat((out1, out2), axis=1)
print(combined_out.shape)

In [ ]:
cnn_out_h = F.max_pooling_nd(combined_out, ksize=9, stride=10, pad=9//2)
print(cnn_out_h.shape)

In [ ]:
highway_test = L.Highway()

In [ ]:
highway_test(combined_out).shape

In [ ]:
model.highway

In [ ]:
MODEL_TYPE == MODEL_CNN

In [ ]:
model.cnn_out_dim

In [ ]:
cnn_k_widths

In [ ]:
cnn_filters, model.max_pool_stride

In [ ]:
model.cnns

In [ ]:
xp.expand_dims(sp_1,axis=0)

In [ ]:
np.prod(buckets[0]["X_fwd"].shape) * buckets[0]["X_fwd"].itemsize

In [ ]:
bucket_data_fname